<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## How to identify markets that could disrupt the US?

In this notebook, we will be looking for markets that are outpacing supply growth nationwide to look for the needle in the haystack on markets changing faster than the US. We will look for the following criteria:
- Markets with a large, trending skew in supply & demand growth where supply is substantially outpacing demand
- Markets with signals for motivated sellers, specifically looking at the ratio of all inventory experiencing price drops
- Markets that appreciated significantly since COVID, yet have not given back any of those price gains

The notebook is broken up into the following sections:
1. [Import required packages and set up the Parcl Labs API key](#1.-Import-required-packages-and-set-up-the-Parcl-Labs-API-key)
2. [Search for markets](#2.-Search-for-markets)
3. [Get the data](#3.-Get-the-data)
4. [Initial data preparation](#4.-Initial-data-preparation)
5. [Supply & demand skew](#5.-Supply-&-demand-skew)
6. [New construction impact on supply](#6.-New-construction-impact-on-supply)
7. [Active supply price drops](#7.-Active-supply-price-drops)
8. [Appreciation since COVID](#8.-Appreciation-since-COVID)
9. [Real time price check](#9.-Real-time-price-check)

#### What will you create in this notebook?

##### Understanding inflection points in supply and demand

<p align="center">
  <img src="../../../images/YoY Bar Supply Demand.png" alt="Alt text">
</p>

##### Understanding shift in price cuts for on market inventory

<p align="center">
  <img src="../../../images/price_drops.png" alt="Alt text">
</p>

##### Understanding the impact of new construction for on market supply

<p align="center">
  <img src="../../../images/new_construction_supply.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/experimental/supply_and_demand/markets_that_could_disrupt.ipynb)

### 1. Import required packages and setup the Parcl Labs API key

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.ts_stats import TimeSeriesAnalysis
from parcllabs.beta.charting.utils import create_labs_logo_dict
from parcllabs.beta.charting.default_charts import create_dual_axis_chart
from parcllabs.beta.charting.styling import default_style_config as style_config


client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for markets

In [3]:
# get us benchmark
metros = client.search.markets.retrieve(
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    location_type='CBSA',
    limit=300 # get top 300 metros based on population
)

# add us national to comp markets against national numbers
us = client.search.markets.retrieve(
    query='United States',
    limit=1
)

markets = pd.concat([metros, us])

In [4]:
# Metro parcl id
market_parcl_ids = markets['parcl_id'].tolist()

### 3. Retrieve the Data

In [5]:
# Lets get YoY changes for tampa market -- warning, you need a starter account to do this work. 
# this will pull back a lot of data and use a lot of credits. We are 
# capturing weekly supply numbers for 100 metros across the country. 

start_date='2022-09-01'

supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

demand = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

prices = client.market_metrics.housing_event_prices.retrieve(
    parcl_ids=market_parcl_ids,
    start_date='2019-01-01',
    limit=200
)

|████████████████████████████████████████| 301/301 [100%] in 24.3s (12.39/s) 
|████████████████████████████████████████| 301/301 [100%] in 23.2s (12.96/s) 
|████████████████████████████████████████| 301/301 [100%] in 34.1s (8.82/s) 


In [6]:
# get supply side of the market
supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=market_parcl_ids,
    start_date=start_date,
    limit=200
)

|████████████████████████████████████████| 301/301 [100%] in 24.5s (12.26/s) 


In [7]:
# get price changing dynamics
price_changes = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    parcl_ids=market_parcl_ids,
    limit=200,
    start_date=start_date
)

|████████████████████████████████████████| 301/301 [100%] in 1:08.6 (4.39/s) 


### 4. Initial data preparation

In [8]:
# supply is a weekly series while demand is a monthly series, need to truncate supply to the month
# and then compress it to a monthly series. We will take the median value of the month to represent
# that month
supply_monthly = supply.copy(deep=True)
supply_monthly = supply_monthly.merge(price_changes[['parcl_id', 'date', 'count_price_drop']], on=['parcl_id', 'date'])
supply_monthly['pct_price_drops'] = supply_monthly['count_price_drop']/supply_monthly['for_sale_inventory']
supply_monthly['date'] = supply_monthly['date'].dt.to_period('M').dt.to_timestamp()
supply_monthly = supply_monthly.groupby(['date', 'parcl_id'])[['for_sale_inventory', 'pct_price_drops']].median().reset_index()


In [9]:
# now we can join with demand, which is a monthly series
data = pd.merge(demand[['date', 'parcl_id', 'sales']], supply_monthly[['date', 'parcl_id', 'for_sale_inventory']], on=['date', 'parcl_id'])
data

,date,parcl_id,sales,for_sale_inventory
0,2024-05-01,2900187,18573,27381.0
1,2024-04-01,2900187,20806,26028.0
2,2024-03-01,2900187,20148,24117.0
3,2024-02-01,2900187,18694,21852.0
4,2024-01-01,2900187,20935,16994.0
...,...,...,...,...
6316,2023-01-01,5826765,565991,513426.0
6317,2022-12-01,5826765,662952,541343.0
6318,2022-11-01,5826765,665793,600837.5
6319,2022-10-01,5826765,749013,595809.0


### 5. Supply & demand skew

In [10]:
data = data.sort_values(['parcl_id', 'date'])
# Calculate percentage change in 'sales' for each 'parcl_id' over 12 periods
data['pct_change_demand'] = data.groupby('parcl_id')['sales'].pct_change(periods=12)
# Calculate percentage change in 'for_sale_inventory' for each 'parcl_id' over 12 periods
data['pct_change_supply'] = data.groupby('parcl_id')['for_sale_inventory'].pct_change(periods=12)
# calculate a 3 month moving average to account for spikiness in the data
data['ma_pct_change_demand'] = data.groupby('parcl_id')['pct_change_demand'].transform(lambda x: x.rolling(window=3).mean())
# Calculate a 3-month moving average for 'pct_change_supply'
data['ma_pct_change_supply'] = data.groupby('parcl_id')['pct_change_supply'].transform(lambda x: x.rolling(window=3).mean())

In [11]:
# clean up the data due to needing 12 observations to account for starting the percent change over a 1 year period
supply_demand_imbalance = data.dropna()

In [32]:
markets['state'] = markets['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0])
markets['clean_name'] = markets.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
markets['clean_name'] = markets['clean_name'].replace({'United States Of America, UNITED STATES OF AMERICA': 'USA'})

In [33]:
# Assuming supply_demand_imbalance and metros dataframes are already defined
supply_demand_imbalance_last = supply_demand_imbalance.loc[supply_demand_imbalance['date'] == '5/1/2024']
supply_demand_imbalance_last = pd.merge(supply_demand_imbalance_last, markets[['parcl_id', 'clean_name', 'state']], on='parcl_id')
supply_demand_imbalance_last_us = supply_demand_imbalance_last.loc[supply_demand_imbalance_last['parcl_id']==us['parcl_id'].values[0]]
supply_demand_imbalance_last = supply_demand_imbalance_last.loc[
    (supply_demand_imbalance_last['sales'] > 500) & 
    (supply_demand_imbalance_last['for_sale_inventory'] > 500) & 
    (supply_demand_imbalance_last['ma_pct_change_demand'] < -0.1) & 
    (supply_demand_imbalance_last['ma_pct_change_supply'] > 0.2)
]
supply_demand_imbalance_last = pd.concat([supply_demand_imbalance_last_us, supply_demand_imbalance_last])
# capture parcl_ids for later analysis
imbalanced_parcl_ids = supply_demand_imbalance_last['parcl_id'].unique().tolist()

In [155]:
markets.loc[markets['parcl_id'].isin(imbalanced_parcl_ids)][['clean_name', 'parcl_id']]

,clean_name,parcl_id
3,"Dallas, TX",2899734
7,"Miami, FL",2900128
8,"Atlanta, GA",2887280
10,"Phoenix, AZ",2900245
17,"Tampa, FL",2900417
20,"St. Louis, MO",2900321
21,"Orlando, FL",2900213
29,"Cincinnati, OH",2899647
30,"Kansas City, MO",2900012
49,"Birmingham, AL",2899611


In [34]:
supply_demand_imbalance_last.loc[supply_demand_imbalance_last['parcl_id']==us['parcl_id'].values[0]]

,date,parcl_id,sales,for_sale_inventory,pct_change_demand,pct_change_supply,ma_pct_change_demand,ma_pct_change_supply,clean_name,state
300,2024-05-01,5826765,600617,1020034.5,-0.216453,0.195957,-0.144631,0.186154,USA,UNITED STATES OF AMERICA


In [156]:
# Add a column to identify selected states
target_states = {'FL'}
supply_demand_imbalance_last['color_group'] = supply_demand_imbalance_last['state'].apply(lambda x: 'Florida' if x in target_states else 'Other')

# Creating the scatter plot
fig = px.scatter(
    supply_demand_imbalance_last, 
    x='ma_pct_change_demand', 
    y='ma_pct_change_supply', 
    color='color_group',  # Use the new color_group column for color
    hover_name='clean_name', 
    title='YoY Changes in Supply vs. Demand (May `24)',
    color_discrete_map={'Florida': 'red', 'Other': 'blue'},  # Customize colors,
    text='clean_name'
)

fig.update_traces(
    textposition='top center',
    mode='markers+text'  # Ensure that both markers and text are displayed
)

fig.add_layout_image(
        create_labs_logo_dict()
    )

# Update axes labels and layout to format as a square
fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
     xaxis=dict(
            title_text='YoY % Change Demand (Sales)',
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            # tickangle=style_config['tick_angle'],
            tickformat='.0%',
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
        yaxis=dict(
            title_text='YoY % Change Supply',
            showgrid=style_config['showgrid'],
            gridwidth=style_config['gridwidth'],
            gridcolor=style_config['grid_color'],
            tickfont=style_config['axis_font'],
            zeroline=False,
            tickformat='.0%',
            linecolor=style_config['line_color_axis'],
            linewidth=style_config['linewidth'],
            titlefont=style_config['title_font_axis']
        ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1000,
    height=1000,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
            x=style_config['legend_x'],
            y=style_config['legend_y'],
            xanchor=style_config['legend_xanchor'],
            yanchor=style_config['legend_yanchor'],
            font=style_config['legend_font'],
            bgcolor='rgba(0, 0, 0, 0)'
        ),
)

fig.show()


In [164]:
# Sort the data by supply percentage change
supply_demand_imbalance_last = supply_demand_imbalance_last.sort_values(by='ma_pct_change_supply', ascending=True)

# Prepare the data for the bar chart
data_for_bar = pd.melt(supply_demand_imbalance_last, id_vars=['clean_name'], 
                       value_vars=['ma_pct_change_demand', 'ma_pct_change_supply'], 
                       var_name='type', value_name='percent_change')
data_for_bar['type'] = data_for_bar['type'].map({'ma_pct_change_demand': 'Demand', 'ma_pct_change_supply': 'Supply'})

# Apply bold formatting and light blue color to markets ending with "FL"
data_for_bar['clean_name'] = data_for_bar['clean_name'].apply(lambda x: f"<b style='color:red'>{x}</b>" if x.endswith('FL') else x)

# Create the bar chart
fig = px.bar(data_for_bar, 
             x='clean_name', 
             y='percent_change', 
             color='type', 
             barmode='relative', 
             title='YoY Change in Supply and Demand (May, `24)',
             labels={'percent_change': 'Percent Change', 'clean_name': 'Market'},
             color_discrete_map={'Demand': 'red', 'Supply': 'green'})

# Update the legend names
for trace in fig.data:
    if trace.name == 'Demand':
        trace.name = 'Demand (Sales)'
    elif trace.name == 'Supply':
        trace.name = 'Supply (Inventory)'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='Percent Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())

fig.show()


### 6. New construction impact on supply

In [167]:
# event counts for new listings
new_listings = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=imbalanced_parcl_ids,
    limit=1
)

# we will need to secure data from 3 separate endpoints
nc_listings = client.new_construction_metrics.housing_event_counts.retrieve(
    parcl_ids=imbalanced_parcl_ids,
    limit=1
)

# rename new_listings_for_sale to new_construction
nc_listings = nc_listings.rename(columns={
    'new_listings_for_sale': 'new_construction_new_listings_for_sale'
})

|████████████████████████████████████████| 44/44 [100%] in 3.8s (11.46/s) 
|████████████████████████████████████████| 44/44 [100%] in 3.5s (12.70/s) 


In [168]:
new_listings = new_listings.merge(nc_listings[['parcl_id', 'new_construction_new_listings_for_sale']], on='parcl_id')
new_listings['pct_new_construction'] = new_listings['new_construction_new_listings_for_sale']/new_listings['new_listings_for_sale']

new_listings = new_listings.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

In [169]:
# Prepare the data for the bar chart
new_listings = new_listings.sort_values('pct_new_construction', ascending=True)
data_for_bar = pd.melt(new_listings, id_vars=['clean_name'], 
                       value_vars=['pct_new_construction'], 
                       var_name='type', value_name='percentage')
data_for_bar['type'] = data_for_bar['type'].map({'pct_new_construction': 'New Construction'})
data_for_bar = data_for_bar.loc[data_for_bar['type']=='New Construction']

# Apply bold formatting and red color to markets ending with "FL"
data_for_bar['clean_name'] = data_for_bar['clean_name'].apply(lambda x: f"<b style='color:red'>{x}</b>" if x.endswith('FLZ') else x)

# Create the stacked bar chart
fig = px.bar(data_for_bar, 
             x='clean_name', 
             y='percentage', 
             color='type', 
             barmode='stack', 
             title='Percent of New Listings Coming from New Construction (May, `24)',
             labels={'percentage': 'Percentage', 'clean_name': 'Market'},
             color_discrete_map={'New Construction': 'orange', 'Investors': 'blue'})

# Update the legend names
for trace in fig.data:
    if trace.name == 'New Construction':
        trace.name = 'New Construction'
    elif trace.name == 'Investors':
        trace.name = 'Investors'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='% of New Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())

fig.show()

### 7. Active supply price drops

Now within these skewed markets, which markets also are having price changes? These markets would now have not only a supply/demand skew but also a supply side that is demonstrating a willingness to sell, actively reducing prices. 

Let's look for markets where at least 35% of the inventory is experiencing price changes.

In [58]:
supply_monthly['ma_price_changes'] = supply_monthly.groupby('parcl_id')['pct_price_drops'].transform(lambda x: x.rolling(window=3).mean())
s1 = supply_monthly.loc[(supply_monthly['ma_price_changes']>0.35) & (supply_monthly['date']=='5/1/2024')].sort_values('ma_price_changes', ascending=False)
s1 = s1.loc[s1['parcl_id'].isin(imbalanced_parcl_ids)]
imbalanced_with_price_changes_pids = s1['parcl_id'].unique().tolist()

In [154]:
# markets left
markets.loc[markets['parcl_id'].isin(imbalanced_with_price_changes_pids)][['clean_name']]

,clean_name
7,"Miami, FL"
10,"Phoenix, AZ"
17,"Tampa, FL"
21,"Orlando, FL"
69,"North Port, FL"
80,"Lakeland, FL"
89,"Deltona, FL"
95,"Palm Bay, FL"
110,"Myrtle Beach, SC"
111,"Port St. Lucie, FL"


In [162]:
price_changes_skewed = price_changes.loc[price_changes['parcl_id'].isin(imbalanced_with_price_changes_pids + [5826765])].merge(supply[['parcl_id', 'date', 'for_sale_inventory']], on=['parcl_id', 'date'])
price_changes_skewed['pct_price_drops'] = price_changes_skewed['count_price_drop']/price_changes_skewed['for_sale_inventory']
price_changes_skewed = price_changes_skewed.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

# Create the line chart using Plotly Express
fig = px.line(
    price_changes_skewed,
    x='date',
    y='pct_price_drops',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_price_drops': '% of Inventory with Price Cuts'},
    title='Percentage of Inventory with Price Reductions (June 17, `24)'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(price_changes_skewed['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = price_changes_skewed[
        (price_changes_skewed['clean_name'] == trace.name) &
        (price_changes_skewed['date'] == latest_date)
    ]['pct_price_drops'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.01 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Price Reductions',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()


### 8. Appreciation since COVID

In [83]:
# filter to most out of balance markets regarding supply and demand
prices_skew = prices.loc[prices['parcl_id'].isin(imbalanced_with_price_changes_pids + [5826765])]

all_rows = []
for pid in prices_skew['parcl_id'].unique().tolist():
    prices_skew_test = prices_skew.loc[prices_skew['parcl_id']==pid]
    price_ts_analysis = TimeSeriesAnalysis(prices_skew_test, 'date', 'price_per_square_foot_median_sales', freq='M')
    price_rate_of_change_stats = price_ts_analysis.calculate_changes(change_since_date='3/1/2020')
    row = pd.json_normalize(price_rate_of_change_stats)
    row['parcl_id'] = pid
    all_rows.append(row)

In [84]:
ts_analysis = pd.concat(all_rows)

In [85]:
hf = ts_analysis.loc[(ts_analysis['peak_to_current.percent_change'] > -.05) & (ts_analysis['change_since_date.percent_change']>.5)]

In [114]:
# markets left
hf[['parcl_id' ,'peak_to_current.percent_change', 'change_since_date.percent_change']].merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

,parcl_id,peak_to_current.percent_change,change_since_date.percent_change,clean_name
0,2900128,-0.0042,0.6373,"Miami, FL"
1,2900417,-0.0092,0.6655,"Tampa, FL"
2,2900213,-0.0031,0.5610,"Orlando, FL"
3,2900041,-0.0463,0.5136,"Lakeland, FL"
4,2899748,-0.0196,0.5546,"Deltona, FL"
5,2900229,0.0000,0.9380,"Palm Bay, FL"
6,2900170,-0.0106,0.5665,"Myrtle Beach, SC"
7,2900268,-0.0049,0.6550,"Port St. Lucie, FL"
8,2900173,0.0000,0.8904,"Naples, FL"
9,2900198,-0.0090,0.6038,"Ocala, FL"


In [120]:
hf_ids = hf['parcl_id'].unique().tolist()
prices_skew = prices.loc[prices['parcl_id'].isin(hf_ids + [5826765])]


In [121]:
# show percent change relative to first value
prices_skew_chart = prices_skew.loc[prices_skew['date']>='3/1/2020']
prices_skew_chart = prices_skew_chart.sort_values('date')
prices_skew_chart = prices_skew_chart[['date', 'parcl_id', 'price_per_square_foot_median_sales']]
prices_skew_chart_first = prices_skew_chart.loc[prices_skew_chart['date']=='3/1/2020'].rename(columns={'price_per_square_foot_median_sales':'start'})
chart = pd.merge(prices_skew_chart, prices_skew_chart_first[['parcl_id', 'start']], on='parcl_id')
chart['pct_change'] = (chart['price_per_square_foot_median_sales']-chart['start'])/chart['start']
chart = chart.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')

In [158]:
# Create the line chart using Plotly Express
fig = px.line(
    chart,
    x='date',
    y='pct_change',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_change': '% Change'},
    title='% Change in Home Values since the Start of the Pandemic (May `24)'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(chart['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = chart[
        (chart['clean_name'] == trace.name) &
        (chart['date'] == latest_date)
    ]['pct_change'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()


### 9. Real time price check

In [141]:
pf_markets = client.search.markets.retrieve(
    sort_by='PRICEFEED_MARKET',
    limit=100,
)

pf_ids = pf_markets.loc[pf_markets['parcl_id'].isin(hf_ids)]['parcl_id'].unique().tolist()
pf_ids

[2899729, 2900213, 2900417, 2899748, 2900128, 2900041]

In [153]:
# markets left
markets.loc[markets['parcl_id'].isin(pf_ids)][['clean_name', 'parcl_id']]

,clean_name,parcl_id
7,"Miami, FL",2900128
17,"Tampa, FL",2900417
21,"Orlando, FL",2900213
80,"Lakeland, FL",2900041
89,"Deltona, FL",2899748
171,"Crestview, FL",2899729


In [142]:
START_DATE = '2020-03-01'
sales_price_feeds = client.price_feed.price_feed.retrieve(
    parcl_ids=pf_ids,
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

|████████████████████████████████████████| 6/6 [100%] in 1.8s (3.37/s) 


In [143]:
sales_price_feeds.head()

,date,price_feed,parcl_id
0,2024-06-24,251.03,2899729
1,2024-06-23,252.39,2899729
2,2024-06-22,253.65,2899729
3,2024-06-21,254.68,2899729
4,2024-06-20,256.08,2899729


In [149]:
sales_price_feeds = sales_price_feeds.sort_values('date')
sales_price_feeds = sales_price_feeds[['date', 'parcl_id', 'price_feed']]
sales_price_feeds_first = sales_price_feeds.loc[sales_price_feeds['date']=='3/1/2020'].rename(columns={'price_feed':'start'})
chart_pf = pd.merge(sales_price_feeds, sales_price_feeds_first[['parcl_id', 'start']], on='parcl_id')
chart_pf['pct_change'] = (chart_pf['price_feed']-chart_pf['start'])/chart_pf['start']
chart_pf = chart_pf.merge(markets[['parcl_id', 'clean_name']], on='parcl_id')
chart_pf

,date,parcl_id,price_feed,start,pct_change,clean_name
0,2020-03-01,2900417,135.09,135.09,0.000000,"Tampa, FL"
1,2020-03-01,2900128,193.15,193.15,0.000000,"Miami, FL"
2,2020-03-01,2899748,137.73,137.73,0.000000,"Deltona, FL"
3,2020-03-01,2900213,144.86,144.86,0.000000,"Orlando, FL"
4,2020-03-01,2899729,157.45,157.45,0.000000,"Crestview, FL"
...,...,...,...,...,...,...
9457,2024-06-24,2900417,236.00,135.09,0.746983,"Tampa, FL"
9458,2024-06-24,2900213,229.06,144.86,0.581251,"Orlando, FL"
9459,2024-06-24,2899748,213.70,137.73,0.551586,"Deltona, FL"
9460,2024-06-24,2900128,328.57,193.15,0.701113,"Miami, FL"


In [160]:
chart_pf[['clean_name', 'parcl_id']].drop_duplicates()

,clean_name,parcl_id
0,"Tampa, FL",2900417
1,"Miami, FL",2900128
2,"Deltona, FL",2899748
3,"Orlando, FL",2900213
4,"Crestview, FL",2899729
5,"Lakeland, FL",2900041


### 5. Chart the Data

In [159]:
# Create the line chart using Plotly Express
# chart_pf = chart_pf.loc[chart_pf['date']>='1/1/2024']
fig = px.line(
    chart_pf,
    x='date',
    y='pct_change',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_change': '% Change'},
    title='% Change in Home Values since the Start of the Pandemic (June 24, `24)'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(chart_pf['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = chart_pf[
        (chart_pf['clean_name'] == trace.name) &
        (chart_pf['date'] == latest_date)
    ]['pct_change'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=1600,
    height=800,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

fig.show()
